In [11]:
import pandas as pd

from pre_process_dataset import PreProcess
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import RandomizedSearchCV

In [2]:
df_train = pd.read_csv("data/medical_tc_train.csv")
df_test = pd.read_csv("data/medical_tc_test.csv")
df_train.loc[df_train['condition_label'] == 5, 'condition_label'] = 0.0
df_test.loc[df_test['condition_label'] == 5, 'condition_label'] = 0.0
print("The total number of training data is: %d" %(df_train.shape[0]))
print("The total number of testing data is: %d" %(df_test.shape[0]))

The total number of training data is: 11550
The total number of testing data is: 2888


In [3]:
df = pd.concat([df_train, df_test], ignore_index=True)
print(f"✅ Combined dataset size: {df.shape[0]} samples")

✅ Combined dataset size: 14438 samples


In [4]:
clean_df = PreProcess(df)
df = clean_df.preprocess_dataset()
df.head(1)

,condition_label,medical_abstract
0,0,tissue change around loose prosthesis canine m...


In [5]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df['medical_abstract'])
y = df['condition_label']

In [6]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


print(f"Training set size: {X_train.shape[0]}")
print(f"Validation set size: {X_test.shape[0]}")
print(f"Test set size: {X_valid.shape[0]}")

Training set size: 10106
Validation set size: 2166
Test set size: 2166


In [7]:
# Scale features
scaler = StandardScaler(with_mean=False)
X_train = scaler.fit_transform(X_train)
X_valid = scaler.fit_transform(X_valid)
X_test = scaler.transform(X_test)

y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)
y_test = to_categorical(y_test)

In [12]:
def create_model(learning_rate=0.001, dropout=0.3, hidden_units=32):
    model = Sequential([
        Dense(hidden_units, activation="relu", input_shape=(X_train.shape[1],)),
        Dropout(dropout),
        Dense(5, activation="softmax"),
    ])
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

model = KerasClassifier(
    model=create_model,
    epochs=40,
    batch_size=16,
    verbose=0,
    callbacks=[EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)]
)

param_dist = {
    "model__learning_rate": [0.01, 0.001, 0.0005],
    "model__dropout": [0.2, 0.3, 0.4, 0.5],
    "model__hidden_units": [16, 32, 64, 128],
    "batch_size": [8, 16, 32],
    "epochs": [10, 30, 50]
}

search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=5,    # only 5 trials to save CPU
    cv=3,
    verbose=1,
    n_jobs=-1
)

search.fit(X_valid, y_valid, validation_split=0.2)

# -------------------------------
# 6. Evaluate best model
# -------------------------------
print("\nBest hyperparameters:")
print(search.best_params_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/home/dip07/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-11-10 17:06:38.050388: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/dip07/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/dip07/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` ar


Best hyperparameters:
{'model__learning_rate': 0.001, 'model__hidden_units': 128, 'model__dropout': 0.5, 'epochs': 30, 'batch_size': 8}


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers


input_dim = X_train.shape[1]

model = Sequential([
    Dense(128, input_dim=input_dim, kernel_regularizer=regularizers.l2(0.001), activation='relu'),
    Dropout(0.5),
    # Dense(64, kernel_regularizer=regularizers.l2(0.001), activation='relu'),
    # Dropout(0.2),
    Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=1)


Epoch 1/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - accuracy: 0.5066 - loss: 1.6596
Epoch 2/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 23s 36ms/step - accuracy: 0.6945 - loss: 1.3744
Epoch 3/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - accuracy: 0.7333 - loss: 1.3294
Epoch 4/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 20s 32ms/step - accuracy: 0.7566 - loss: 1.3155
Epoch 5/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.7620 - loss: 1.3385
Epoch 6/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 26s 41ms/step - accuracy: 0.7676 - loss: 1.3392
Epoch 7/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.7612 - loss: 1.3767
Epoch 8/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.7578 - loss: 1.4344
Epoch 9/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 25s 40ms/step - accuracy: 0.7616 - loss: 1.4429
Epoch 10/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - accuracy: 0.7593 - loss: 1.4575


In [16]:
loss, acc = model.evaluate(X_test, y_test)
print("Accuracy:", acc)

68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5032 - loss: 2.0565
Accuracy: 0.5032317638397217


In [ ]:
model = Sequential([
    Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01),
          input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(5, activation='softmax')
])

# Compile
model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# -------------------------------
# 4. Early stopping
# -------------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -------------------------------
# 5. Train the model
# -------------------------------
model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=8,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# -------------------------------
# 6. Evaluate on test set
# -------------------------------
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest accuracy: {test_acc:.3f}")

Epoch 1/100


ValueError: Argument `output` must have rank (ndim) `target.ndim - 1`. Received: target.shape=(None, 5), output.shape=(None, 5)